# Training an entities recognition model

Importing the required code files

In [1]:
from os import getcwd, path
import sys
import matplotlib.pyplot as plt

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

from entities_recognition.bilstm.train import trainIters, evaluate
from config import START_TAG, STOP_TAG

The training data must be an array that:
- Contains tuples of (sentence, tags)
- Sentence will be splitted using nltk.wordpunct_tokenize
- Tags will be splitted using .split() - hence spaces by default

Each entity must be separated into 3 kinds of tag: B- (Begin), I- (Inside) and O- (Outside)

_This is to help with separation in the case of consecutive entities_

A `dictionary` to translate from these tags into consecutive indices must be defined
This dictionary will contain:
- The empty token
- `START_TAG` and `END_TAG` tokens (imported from global configs - used internally to indicate start and end of sentence)
- Entities B-, I-, O- tokens

**Sample training data for email recognition:**

In [2]:
training_data = [(
    'My email address is at luungoc2005@gmail.com',
    '- - - - - - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL'
), (
    'Contact me at contact@2359media.net',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL'
), (
    'test.email@microsoft.com is a testing email address',
    'B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL - - - - - - - - - -'
), (
    'Any inquiries email thesloth_197@gmail.com for assistance',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL - - - -'
), (
    'Email addresses include test.noreply@gmail.com hello.vietnam@hallo.org contact@rocket.net',
    '- - - - - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL - B-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL - B-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL'
), (
    'Contact: tester@github.com at any hours',
    '- - - B-EMAIL I-EMAIL I-EMAIL I-EMAIL O-EMAIL - - - - - -'
)]

tag_to_ix = {
    '-': 0,
    'B-EMAIL': 1,
    'I-EMAIL': 2,
    'O-EMAIL': 3,
    START_TAG: 4,
    STOP_TAG: 5
}

Begin training the network
Logs will be saved into `entities_recognition/bilstm/logs` by default

Run `tensorboard --logdir=entities_recognition/bilstm/logs` from the root directory for training logs

Verbosity:
- `verbose = 0` for almost no console output
- `verbose = 1` will only log on `log_every` (10 epochs by default)
- `verbose = 2` (default) will use tqdm for both loops

In this case ~50 epochs should be sufficient (found by trial and error)

In [3]:
model = trainIters(training_data, tag_to_ix, n_iters=50, log_every=5, verbose=1)

Importing /Users/2359media/Documents/botbot-nlp/data/glove/glove.6B.300d.pickle...
0m 56s (- 8m 32s) (5 10%) 117.7349
1m 2s (- 4m 11s) (10 20%) 20.0381
1m 8s (- 2m 40s) (15 30%) 3.1384
1m 14s (- 1m 52s) (20 40%) 1.2917
1m 20s (- 1m 20s) (25 50%) 0.5144
1m 26s (- 0m 57s) (30 60%) 0.3027
1m 32s (- 0m 39s) (35 70%) 0.2321
1m 38s (- 0m 24s) (40 80%) 0.1796
1m 44s (- 0m 11s) (45 90%) 0.1354
1m 50s (- 0m 0s) (50 100%) 0.1027


Evaluate model accuracy by using

In [4]:
evaluate(model, training_data, tag_to_ix)

1.0